## **3. Pondération statistique** (TF-IDF)  

https://stackoverflow.com/questions/46580932/calculate-tf-idf-using-sklearn-for-n-grams-in-python  
http://scikit-learn.sourceforge.net/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn-feature-extraction-text-tfidfvectorizer  
https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction


In [1]:
path = '/Users/camilledemers/Documents/05-transformation/'
acteur = 'inspq'
tag = 'agression-sexuelle'

In [3]:
import pandas as pd 

file_path = path + tag + '/' + tag + '_terms.csv'
with open(file_path) as f:
    csv = pd.read_csv(f)

In [4]:
vocabulaire = list(csv['Terme'])

In [5]:
# Importer les données (fichiers .txt)
import os

def lire_corpus(acteur, tag):
    path = '//Users/camilledemers/Documents/03-corpus/2-sous-corpus/' + acteur + '/' + tag + '/'

    corpus =  [] 
    corpus_index = []

    # Change the directory
    os.chdir(path)

    for file in os.listdir()[:300]: # les 300 premiers documents seulement 
        if file.endswith(".txt"):
          file_path = path + file
        
        with open(file_path, 'r', encoding = "UTF-8") as f:
            try:
                data = f.readlines()
                corpus.append(data[1].lower().strip('\n'))
            
            except:
                True

    return corpus

corpus = lire_corpus(acteur, tag)

print("On a un corpus de {} documents".format(len(corpus)))

On a un corpus de 44 documents


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(vocabulary = vocabulaire, stop_words=None, ngram_range=(2,8))
tfs = tfidf.fit_transform(corpus)

In [7]:
features_names = tfidf.get_feature_names_out()
corpus_index = [corpus.index(n) for n in corpus]

import pandas as pd
df = pd.DataFrame(tfs.T.todense(), index=features_names, columns=corpus_index)

In [11]:
path = '/Users/camilledemers/Documents/04-filtrage/' + acteur + '/'
file_path = path + tag + '/' +  tag + '_matrice-TFIDF.csv'
df.to_csv(file_path)

In [13]:
terms_weighted = []
rows, cols = tfs.nonzero()
for row, col in zip(rows,cols):
    terms_weighted.append([features_names[col], tfs[row,col]])

In [15]:
terms_weighted = pd.DataFrame(terms_weighted, columns=['Terme', 'TF-IDF'])
terms_weighted.sort_values(["TF-IDF"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)
terms_weighted.to_csv(path + '/' + tag + '/' + tag + '_weighting_TF-IDF.csv')

In [142]:
terms_weighted

,Terme,TF-IDF
343490,violence conjugale,0.981352
430781,inspq qc,0.723175
783743,inspq qc,0.658864
430869,assurance qualité,0.644915
783742,qc ca publications auteurs,0.520918
...,...,...
247062,institut abonnez,0.003698
247061,derniers numéros,0.003698
447306,derniers numéros,0.003652
447353,répertoire des analyses,0.003652
